In [1]:
!pip install dash==1.19.0 
!pip install jupyter_dash 
!pip install --upgrade plotly


  Attempting uninstall: plotly
    Found existing installation: plotly 5.3.1
    Uninstalling plotly-5.3.1:
      Successfully uninstalled plotly-5.3.1


In [4]:
# Import required libraries
import pandas as pd 
import dash 
import dash_html_components as html 
import dash_core_components as dcc 
from dash.dependencies import Input, Output 
from jupyter_dash import JupyterDash 
import plotly.graph_objects as go
import plotly.express as px

In [5]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload

9600.0

In [6]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [7]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [8]:
spacex_df.columns

Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'Mission Outcome',
       'class', 'Payload Mass (kg)', 'Booster Version',
       'Booster Version Category'],
      dtype='object')

In [10]:
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
# TASK 1: Add a dropdown list to enable Launch Site selection
# The default select value is for ALL sites
# dcc.Dropdown(id='site-dropdown',...)
dcc.Dropdown(id='site-dropdown',
    options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            ],
    value='ALL',
    placeholder="Select a Launch Site",
    searchable=True
     ),
html.Br(),

# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
html.Div(dcc.Graph(id='success-pie-chart')),
html.Br(),

html.P("Payload range (Kg):"),
# TASK 3: Add a slider to select payload range
#dcc.RangeSlider(id='payload-slider',...)
dcc.RangeSlider(id='payload-slider',
                min=0, 
                max=10000, 
                step=1000,
                marks={0: '0', 1000: '1000', 2000: '2000', 3000: '3000', 4000: '4000',
                      5000: '5000', 6000:'6000', 7000:'7000', 8000:'8000', 9000:'9000',
                    10000: '10000',
                      },
                value=[min_payload, max_payload]),
                                
# TASK 4: Add a scatter chart to show the correlation between payload and launch success
html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(value):
    filtered_data = spacex_df[spacex_df['class']==1]
    data = filtered_data.groupby('Launch Site')['class'].count().reset_index()
    if value == 'ALL':
        fig = px.pie(data, values='class', 
        names='Launch Site', 
        title='Success vs. Failed Counts for All Sites')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==value]
        pie_data = filtered_df.groupby('class')['Launch Site'].count().reset_index()
        fig = px.pie(pie_data, values='Launch Site', 
        names='class', 
        title='Success vs. Failure Counts for site: ' + value)
        
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),   
    Input(component_id='payload-slider', component_property='value'),
    Input(component_id='site-dropdown', component_property='value'))

def get_scatter_chart(plvalue, ddvalue):
    scatter_df = spacex_df[spacex_df['Payload Mass (kg)'].between(plvalue[0], plvalue[1])]
    scatter_df2 = scatter_df[scatter_df['Launch Site']==ddvalue]
    #data = spacex_df.groupby('Launch Site')['class'].count().reset_index()
    #x_data = spacex_df['Payload Mass (kg)']
   #y_data = spacex_df['class']
    if ddvalue == 'ALL':
        fig = px.scatter(
                scatter_df,
                x='Payload Mass (kg)', 
                y='class',
                color='Booster Version Category',      
                title='Correlation between Payload and Success for All Sites')  
    else:
        fig = px.scatter(
        scatter_df2,
        x='Payload Mass (kg)', 
        y='class',
        color='Booster Version Category',      
        title='Correlation between Payload and Success for Site: ' + ddvalue) 
        
    return fig
# Run the app
if __name__ == '__main__': app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, 
    debug=True, dev_tools_hot_reload =True, threaded=True)